In [ ]:
!pip install mne

import scipy.io
from io import BytesIO
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from mne.decoding import CSP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.0 MB/s eta 0:00:00


In [ ]:
mat = scipy.io.loadmat('Walk.mat')
mat
ecog_data = mat['y']

In [ ]:
paradigmInfo = scipy.io.loadmat('Walk_paradigmInfo.mat')

In [ ]:
with open('Walk_settings.txt') as f:
  settings = f.read()
  print(settings)

fs: 1200/128

CH1: time
CH2-161: ECoG 1-160
CH162: DI (Photodiode Feedback)
CH163: StimCode
CH164: GroupId

no reference


In [ ]:
with open('Walk_paradigm.xml') as f:
  paradigminfo = f.read()
  print(paradigminfo)

<?xml version="1.0" encoding="utf-8"?>
<Data  xmlns="xsd"
           xmlns:ns="http://www.w3.org/2001/XMLSchema-instance"
           ns:schemaLocation="xsd C:\_SVN\ECoGmeToo\trunk\sources\ParadigmTools\src\main\Paradigm\ParadigmSchema.xsd">

  <Paradigm BaseFolder="media/">
    <Task ns:type="SingleTask" ID="ST_PreParadigm" DurationSeconds="10">
      <Stimulus ns:type="TextStimulus" Caption="+"/>
    </Task>
    <Task ns:type="SingleTask" ID="ST_Video" DurationSeconds="252" Group="1">
      <Stimulus ns:type="VideoStimulus" FileName="walk.mp4"/>
    </Task>
    <Task ns:type="SingleTask" ID="ST_PostParadigm" DurationSeconds="5">
      <Stimulus ns:type="TextStimulus" Caption="Thank You!"/>
    </Task>
  </Paradigm>

</Data>


# Data exploration

In [ ]:
labels ={
    'digit': ['0:10-0:17', '0:36-0:41', '3:51-3:54', '3:59-4:02'], 
    'kanji': ['0:27-0:36', '3:42-3:46', '3:54-3:56', '4:02-4:04'],
    'face': ['0:47-0:53', '1:53-2:00', '2:31-2:37', '2:46-2:52', '2:59-3:04', '4:05-4:12'],
    'hira': ['0:56-1:02'],
    "object": ['1:25-1:32', '1:53-1:55', '1:59-2:03', '2:10-2:20', '3:12-3:30', '3:48-4:04'],
    "line": ['1:35-1:40', '3:49-3:51'],
    "body": ['1:48-2:03', '2:26-2:36', '2:53-3:03', '3:32-3:45', '3:56-3:59']
}

In [ ]:
label_channels = np.zeros((7, 322049))
label_channels.shape

(7, 322049)

In [ ]:
for key in labels:

  key_to_idx = {
      'body': 0, 
      'face': 1,
      'digit': 2,
      'hira': 3,
      'kanji': 4,
      'line': 5,
      'object': 6,     
  }

  for span in labels[key]:
    start_time, end_time = span.split('-')
    min, sec = start_time.split(':')
    start_frame = (int(min) * 60 + int(sec) * 1) * 1200
    min, sec = end_time.split(':')
    end_frame = (int(min) * 60 + int(sec) * 1) * 1200
    print(key, start_frame, end_frame)
    label_channels[key_to_idx[key]][start_frame:end_frame] = 1

digit 12000 20400
digit 43200 49200
digit 277200 280800
digit 286800 290400
kanji 32400 43200
kanji 266400 271200
kanji 280800 283200
kanji 290400 292800
face 56400 63600
face 135600 144000
face 181200 188400
face 199200 206400
face 214800 220800
face 294000 302400
hira 67200 74400
object 102000 110400
object 135600 138000
object 142800 147600
object 156000 168000
object 230400 252000
object 273600 292800
line 114000 120000
line 274800 277200
body 129600 147600
body 175200 187200
body 207600 219600
body 254400 270000
body 283200 286800


# Preprocessing steps


### Isolate ECoG data corresponding to video

In [ ]:
# get the indices of the columns where the last row is 1
video_indices = np.where(ecog_data[-1] == 1)[0]
video_data = ecog_data[:, video_indices]
stim_1_data = ecog_data[:, np.where(ecog_data[-2] == 1)[0]]
stim_2_data = ecog_data[:, np.where(ecog_data[-2] == 2)[0]]
stim_3_data = ecog_data[:, np.where(ecog_data[-2] == 3)[0]]

In [ ]:
x = video_data[1:161]
x

array([[ -9394.972  ,  -9312.645  ,  -9271.435  , ...,  -9453.155  ,
         -9442.572  ,  -9445.171  ],
       [ -8803.406  ,  -8740.629  ,  -8708.926  , ...,  -3479.4329 ,
         -3467.3655 ,  -3467.9377 ],
       [-46186.574  , -46110.555  , -46068.73   , ..., -42955.52   ,
        -42940.566  , -42941.434  ],
       ...,
       [  1472.1324 ,   1551.2363 ,   1598.5604 , ...,   -693.6732 ,
          -682.31055,   -686.2936 ],
       [-37829.125  , -37747.676  , -37700.152  , ..., -38397.707  ,
        -38385.68   , -38391.67   ],
       [-64281.17   , -64203.22   , -64156.484  , ..., -64058.324  ,
        -64047.703  , -64049.297  ]], dtype=float32)

In [ ]:
#Implement high pass filter - not needed since no visual inspection done? 
fs = 1200
HP_freq = 

In [ ]:
# Implement bandpass filter to isolate broadband gamma activity 
fs = 1200

#110-140
low_pass_freq = 110
high_pass_freq = 140
filter_order = 4
nyq_freq = 0.5*fs

b, a = signal.butter(filter_order, [low_pass_freq / nyq_freq, high_pass_freq / nyq_freq], btype='bandpass')

x_filt = signal.filtfilt(b,a,x, axis=1)

#If time permits, could also look at lower frequency gamma band (30-100 Hz) but will need to use notch filter to remove power noise (50 or 60 Hz depending on where data was collected)

In [ ]:
# Prepare data to apply Common Spatial Patterns filter

#Extract each trial (exposure to one stimulus) from 100ms to 600ms after exposure and stack in an array + create another vector for the corresponding labels


## Get CLF and apply to data

In [ ]:
# prepped_data: Data needs to be in (trial, channel, sample) format where each trial has the same number of samples 
# Labels need to be in a separate vector of length (trial) with a number corresponding to the type of stimulus (face, letters, digits, etc)

num_classes = len(np.unique(labels))

csp_list = []
lda_list = [] 

for class_number in range(num_classes):
  labels_binary = np.zeros(labels.shape)
  labels_binary[labels == class_number] = 1
  labels_binary[labels != class_number] = -1

  #Fit pipeline to data
  csp = CSP(n_components=4, reg=None, log=None, norm_trace=False)
  lda = LDA()
  clf = Pipeline([('CSP', csp), ('LDA', lda)])
  clf.fit(prepped_data, labels_binary)

  #Save for later
  csp_list.append(csp)
  lda_list.append(lda)


In [ ]:
csp_filtered_data_list = []

for csp in csp_list:
  csp_data = csp.transform(prepped_data)
  csp_filtered_data_list.append(csp_data)

#combine all data
final_data = np.concatenate(csp_filtered_data_list, axis=-1)


In [ ]:
# the labels need to be generned with the filter_data
# count variances
variances = []
for csp_data in final_data:
  variances.append(np.var(window_data))
variances = np.array(variances)

# filter with the variances
new_data, new_label = [],[]
for dat,label in zip(final_data, final_label):
  if np.var(window_data)) > np.percentile(variances, 25):
    new_data.append(dat)
    new_label.append(label)
final_data = np.array(new_data)
final_label = np.array(new_label)



In [ ]:
# train model and find biomarkers

cv = ShuffleSplit(10, test_size=0.2)
clf = LinearDiscriminantAnalysis()
scores = cross_val_score(clf, fina_data, final_labels, cv=cv)


from sklearn import RandomForestClassifier
from xgboost import XGBClassifier
import shap

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
scores = cross_val_score(clf, final_data, final_labels, cv=cv)
clf.fit(epochs_data_train, labels)
importances = clf.feature_importances_


clf = XGBClassifier(n_setimators=100)
scores = cross_val_score(clf, final_data, final_labels, cv=cv)
clf.fit(epochs_data_train, labels)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(final_data)
shap.force_plot(explainer.expected_value, shap_values, final_data)

shap_interaction_values = explainer.shap_interaction_values(final_data)
shap.summary_plot(shap_interaction_values, final_data)

In [ ]:
#IGNORE THIS CELL

#CSP attempt 1 - going to try again using MNE library 

def get_csp(data, labels, n_components): 
  num_classes = len(np.unique(labels))
  #Get class means
  class_means = [np.mean(data[labels == class_num], axis=0) for class_num in np.unique(labels)]

  #Compute spatial covariance matrices for each class
  covs = [np.cov(data[labels == class_num], rowvar=False) for class_num in np.unique(labels)]

  #Get eigenval problem for cov
  _, V = eigh(covs[0], covs[0] + covs[1] + covs[2])
  V = V[:,::-1]     #Sort by descending eigenvalue

  filters = []
  for i in range(num_classes):
      class_i = i
      other_classes = np.setdiff1d(np.arange(num_classes), class_i)
      class_data = data[labels == class_i]
      other_class_data = data[np.isin(labels, other_classes)]
      cov_i = np.cov(class_data.reshape(-1, class_data.shape[-1]), rowvar=False)
      cov_other_classes = np.cov(other_class_data.reshape(-1, other_class_data.shape[-1]), rowvar=False)
      _, V = eigh(cov_i, cov_i + cov_other_classes)
      V = V[:, ::-1]
      W = np.dot(V.T, np.vstack([np.eye(data.shape[1]), np.eye(data.shape[1])]))
      norms = np.sqrt(np.sum(W ** 2, axis=0))
      W /= norms
      filters.append(W)
  return filters